In [3]:
import os
import cv2
import numpy as np
from pathlib import Path

import sys
sys.path.append('../src')  # Add path to import preprocess module
from preprocess import preprocess_image, adjust_boxes, plot_original_and_preprocessed_images


# Preprocess

In [4]:
# Input paths (only training data)
image_dir = '../data/valid/images'
label_dir = '../data/valid/labels'

# Output paths (outside data folder)
output_image_dir = '../enhance_image/valid/images'
output_label_dir = '../enhance_image/valid/labels'

In [5]:
# Create output directories
os.makedirs(output_image_dir, exist_ok=True)
os.makedirs(output_label_dir, exist_ok=True)

# Supported extensions
image_extensions = ('.png', '.jpg', '.jpeg')

In [6]:
# List all images
image_files = sorted([f for f in os.listdir(image_dir) if f.lower().endswith(image_extensions)])


In [7]:
# Helper: read YOLO label
def read_yolo_label(file_path):
    boxes = []
    with open(file_path, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 5:
                cls, cx, cy, w, h = map(float, parts)
                x1 = (cx - w/2) * 416
                y1 = (cy - h/2) * 416
                x2 = (cx + w/2) * 416
                y2 = (cy + h/2) * 416
                boxes.append([x1, y1, x2, y2, int(cls)])
    return boxes

In [8]:
# Helper: write YOLO label
def write_yolo_label(file_path, boxes):
    with open(file_path, 'w') as f:
        for box in boxes:
            x1, y1, x2, y2, cls = box
            cx = ((x1 + x2) / 2) / 416
            cy = ((y1 + y2) / 2) / 416
            w = (x2 - x1) / 416
            h = (y2 - y1) / 416
            f.write(f"{cls} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")


In [9]:
# Get all image files
image_files = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))]

for img_name in sorted(image_files):
    base_name = os.path.splitext(img_name)[0]
    img_path = os.path.join(image_dir, img_name)
    label_path = os.path.join(label_dir, base_name + ".txt")

    if not os.path.exists(label_path):
        print(f"❌ Skipped (no matching label): {img_name}")
        continue

    # Load image and preprocess
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read image: {img_path}")
        continue

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    pre_img, original_size = preprocess_image(img)

    # Read and adjust bounding boxes
    boxes = read_yolo_label(label_path)
    new_boxes = adjust_boxes(boxes, original_size)

    # Save preprocessed image (same name)
    save_img_path = os.path.join(output_image_dir, img_name)
    cv2.imwrite(save_img_path, (pre_img * 255).astype(np.uint8)[:, :, ::-1])  # RGB to BGR

    # Save label (same name)
    save_label_path = os.path.join(output_label_dir, base_name + ".txt")
    write_yolo_label(save_label_path, new_boxes)

    print(f"✅ Processed & saved: {img_name}")

✅ Processed & saved: 00000361_jpg.rf.c348e630be14fc3b53cecec866f8958c.jpg
✅ Processed & saved: 00000379_jpg.rf.1baa78fe909ca826b55000e471624ad9.jpg
✅ Processed & saved: 00000652_jpg.rf.9e9d966620f68afee47a538c42375c45.jpg
✅ Processed & saved: 00000730_jpg.rf.71d7ff1475b0bb162644e0ad8765c341.jpg
✅ Processed & saved: 00000832_jpg.rf.545d3c8cd62de82915e37995d22ba18e.jpg
✅ Processed & saved: 00000833_jpg.rf.d2b30a4cf3c995b4b04473d65ee11838.jpg
✅ Processed & saved: 00000836_jpg.rf.eacc0dfbb5ed048c1f5c6faffc6efede.jpg
✅ Processed & saved: 00000969_jpg.rf.2f949cd1a5a8c9ec57d214937b57a95c.jpg
✅ Processed & saved: 008_derringer_1_jpg.rf.f33f481ca0ccb67484155c93770b2b5b.jpg
✅ Processed & saved: 008_sw10_jpg.rf.fdeceaea05b0cb49abae254ccd77adce.jpg
✅ Processed & saved: 05_14_0000007_jpg.rf.cd1196f4def13841e8fd68e16bb6da80.jpg
✅ Processed & saved: 05_14_0000046_jpg.rf.75f171a2a52bd1c96f2321032cb466fe.jpg
✅ Processed & saved: 05_14_0000051_jpg.rf.6716542dbf10241ec7f8048f766f9319.jpg
✅ Processed & sa